In [ ]:
pip install git+https://github.com/qubvel/segmentation_models.pytorch

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-bpbjp8ng
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-bpbjp8ng
     |████████████████████████████████| 58 kB 2.9 MB/s 
     |████████████████████████████████| 376 kB 16.5 MB/s 
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.2.1-py3-none-any.whl size=88649 sha256=d67d97520babb7e928c4c20e68551c4a6f7d272bef3e23c31e0657d0d2888f93
  Stored in directory: /tmp/pip-ephem-wheel-cache-nirvcagz/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=560b4d3d0ac864c218d0e09b9c38e8f9a6d359d6d944689032a8c27bcbf3732b
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-n

In [ ]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.2 MB/s 


In [ ]:
pip install monai

     |████████████████████████████████| 709 kB 5.2 MB/s 


In [ ]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-vqclz05m
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-vqclz05m
     |████████████████████████████████| 47.6 MB 1.3 MB/s 
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=103681 sha256=44e9a9222450ccc3b402a9f6de60d205391a15a04297706212059904abbecbd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-qvc9nr9j/wheels/63/11/1a/c77caf3ae9b9b6d57b3ee5e6a41a50f3bc12c66a70f6b90bf0
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import logging
import os
import sys
import tempfile
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

from tqdm import tqdm
from pathlib import Path
import time
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import segmentation_models_pytorch as smp
import pydicom
import random

import monai
from monai.optimizers import LearningRateFinder
from monai.inferers import SimpleInferer
from monai.metrics import DiceMetric
import monai.transforms as mtf
from monai.networks import one_hot
from monai.data import decollate_batch
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
arch = "resnet34" # arch : "resnet34", "eff-b0", "eff-b5"
random_windowing = True # random windowing : True - random, False - simple windowing(wl : 40, ww : 400)
pre_valid, fine_valid = range(81,101), range(81,101)
wl, ww = 112,384
batch_size = 24
num_workers = 6
num_epochs = 10
# lr = 0.004

In [ ]:
root_dir = "/content/drive/MyDrive/21_2_Image_process_project/21_2_image_process_png_new"
pth_dir = "/content/drive/MyDrive/21_2_Image_process_project//pth"
submission_name = "submission8.csv"


In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value) 
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars\n
        torch.backends.cudnn.deterministic = True  #needed\n
        torch.backends.cudnn.benchmark = False
        
seed = 42
random_seed(seed,True)

In [ ]:
def path_list(root_dir, mode, valid_num, positive_only = False):
    path_images = []
    path_labels = []
    if mode == "train":
        for patient in tqdm(sorted(os.listdir(os.path.join(root_dir, "train/img")))):

            if patient.split('_')[-1] not in valid_num:
                for z_slice in sorted(os.listdir(os.path.join(root_dir, "train/img", patient))):
                    if z_slice.split('.')[-1] == "png":
                        name = z_slice.split('.')[0]
                        path_image = os.path.join(root_dir, "train/img", patient, f"{name}.png")
                        path_label = os.path.join(root_dir, "train/mask", f"L_{patient.split('_')[-1]}", f"{name}.png")
                        if positive_only:
                            temp_label = cv2.imread(path_label)
                            if np.all(temp_label == 0):
                                continue
                            else:
                                path_images.append(path_image)
                                path_labels.append(path_label)
                        else:
                            path_images.append(path_image)
                            path_labels.append(path_label)
                            
                
    else:
        for patient in tqdm(sorted(os.listdir(os.path.join(root_dir, "valid/img")))):
            if patient.split('_')[-1] in valid_num:
                for z_slice in sorted(os.listdir(os.path.join(root_dir, "valid/img", patient))):
                    if z_slice.split('.')[-1] == "png":
                        name = z_slice.split('.')[0]
                        path_image = os.path.join(root_dir, "valid/img", patient, f"{name}.png")
                        path_label = os.path.join(root_dir, "valid/mask", f"L_{patient.split('_')[-1]}", f"{name}.png")
                        if positive_only:
                            temp_label = cv2.imread(path_label)
                            if np.all(temp_label == 0):
                                continue
                            else:
                                path_images.append(path_image)
                                path_labels.append(path_label)
                        else:
                            path_images.append(path_image)
                            path_labels.append(path_label)
    return path_images, path_labels

In [ ]:
valid_num = []
for i in pre_valid:
    num = format(i, '02')
    valid_num.append(num)

In [ ]:
train_images, train_labels = path_list(root_dir, "train", valid_num, False)
valid_images, valid_labels = path_list(root_dir, "valid", valid_num, False)

100%|██████████| 20/20 [00:04<00:00,  4.97it/s]


In [ ]:
print(f"train images : {len(train_images)}, train labels : {len(train_labels)}")
print(f"valid images : {len(valid_images)}, valid labels : {len(valid_labels)}")


train images : 12400, train labels : 12400
valid images : 3100, valid labels : 3100


In [ ]:
def strong_aug(p=0.5):
    return A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.OneOf([
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=0.1),
        ], p=0.2),
        A.OneOf([
            A.Sharpen(),
            A.RandomBrightnessContrast(),
            A.ElasticTransform()
        ], p=0.3),
        A.OneOf([
            A.Affine(scale = (1.2, 0.8)),
            A.Affine(translate_percent = (0.2, 0.2)),
            A.Affine(rotate = (-30, 30)),
            A.Affine(shear = (-20,20)),
            A.GridDistortion(),
            A.OpticalDistortion(),
        ], p = 0.8),
    ], p=p)
aug_func = strong_aug(p = 0.9)

In [ ]:
train_aug = A.Compose([
    #A.Normalize(mean = 101, std = 76.9, max_pixel_value = 1),
    A.Resize(256,256),
    #A.ToGray(),
    aug_func,
    ToTensorV2(),
])

valid_aug = A.Compose([
    #A.Normalize(mean = 101, std = 76.9, max_pixel_value = 1),
   # A.ToGray(),
    A.Resize(256,256),
    ToTensorV2(),
])

In [ ]:
class MyDataset_train(Dataset):
    global image
    def __init__(self, images_list, labels_list, transform = None):
        self.images_list = images_list
        self.labels_list = labels_list
        self.transform = transform
    def __getitem__(self, idx):
        image = cv2.imread(self.images_list[idx])[:,:,0]
        image = image.astype(np.float32)
        mask = cv2.imread(self.labels_list[idx])[:,:,0]
        
        if self.transform:
            augmented = self.transform(image = image, mask = mask)
            image = augmented['image']
            mask = augmented['mask']
            #print(image.shape)
            mask = mask[np.newaxis,:,:]
        return image, mask
    def __len__(self):
        return len(self.images_list)

class MyDataset_valid(Dataset):
    global image
    def __init__(self, images_list, labels_list, transform = None):
        self.images_list = images_list
        self.labels_list = labels_list
        self.transform = transform
    def __getitem__(self, idx):
        image = cv2.imread(self.images_list[idx])[:,:,0]
        image = image.astype(np.float32)
        mask = cv2.imread(self.labels_list[idx])[:,:,0]
        
        if self.transform:
            augmented = self.transform(image = image, mask = mask)
            image = augmented['image']
            mask = augmented['mask']
            mask = mask[np.newaxis,:,:]
        return image, mask
    def __len__(self):
        return len(self.images_list)

In [ ]:
trainset = MyDataset_train(train_images, train_labels, transform = train_aug)
validset = MyDataset_valid(valid_images, valid_labels, transform = valid_aug)

In [ ]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle = True, num_workers = num_workers)
valid_loader = DataLoader(validset, batch_size=batch_size, shuffle = False, num_workers = num_workers)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
if arch == "resnet34":
    model = smp.Unet(encoder_name = "resnet34", encoder_weights = "imagenet", in_channels = 1, classes = 2)
elif arch == "resnet50":
    model = smp.Unet(encoder_name = "resnet50", encoder_weights = "imagenet", in_channels = 1, classes = 2)
elif arch == "resnet101":
    model = smp.Unet(encoder_name = "resnet101", encoder_weights = "imagenet", in_channels = 1, classes = 2)
elif arch == "eff-b5":
    model = smp.Unet(encoder_name = "timm-efficientnet-b5", encoder_weights = "noisy-student",in_channels = 1, classes = 2)
elif arch == "eff-b0":
    model = smp.Unet(encoder_name = "timm-efficientnet-b0", encoder_weights = "noisy-student",in_channels = 1, classes = 2)


In [ ]:
net = model.to(device)
print(type(net))
net

<class 'segmentation_models_pytorch.unet.model.Unet'>


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
import torch
from torch import nn
from torchsummary import summary as summary_
from torch.nn import functional as F

In [ ]:
summary_(model,(1,256,256),batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           3,136
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
         MaxPool2d-4            [1, 64, 64, 64]               0
            Conv2d-5            [1, 64, 64, 64]          36,864
       BatchNorm2d-6            [1, 64, 64, 64]             128
              ReLU-7            [1, 64, 64, 64]               0
            Conv2d-8            [1, 64, 64, 64]          36,864
       BatchNorm2d-9            [1, 64, 64, 64]             128
             ReLU-10            [1, 64, 64, 64]               0
       BasicBlock-11            [1, 64, 64, 64]               0
           Conv2d-12            [1, 64, 64, 64]          36,864
      BatchNorm2d-13            [1, 64, 64, 64]             128
             ReLU-14            [1, 64,

In [ ]:
model = model.to(device)
# loss_function = monai.losses.DiceCELoss(include_background=False, softmax = True, to_onehot_y = True, ce_weight = torch.tensor([0.3, 0.3,0.4]).cuda())
loss_function = monai.losses.GeneralizedDiceLoss(include_background=False, softmax = True, to_onehot_y = True)
post_trans = mtf.Compose(
    [mtf.Activations(softmax=True)]
)
post_label = mtf.AsDiscrete(to_onehot=True, n_classes=2)
post_pred = mtf.AsDiscrete(argmax=True, to_onehot=True, n_classes=2)
dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
inferer = monai.inferers.SimpleInferer()

/usr/local/lib/python3.7/dist-packages/monai/transforms/post/array.py:177: UserWarning: `to_onehot=True/False` is deprecated, please use `to_onehot=num_classes` instead.
  warnings.warn("`to_onehot=True/False` is deprecated, please use `to_onehot=num_classes` instead.")


In [ ]:
lower_lr, upper_lr = 1e-5, 1e-0
optimizer = torch.optim.Adam(model.parameters(), lower_lr)
lr_finder = LearningRateFinder(model, optimizer, loss_function, device=device)
lr_finder.range_test(train_loader, valid_loader, end_lr=upper_lr, num_iter=5)
steepest_lr, _ = lr_finder.get_steepest_gradient()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Computing optimal learning rate: 100%|██████████| 5/5 [04:21<00:00, 52.24s/it]

Resetting model and optimizer


In [ ]:
print(steepest_lr)

0.056234132519034905


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), steepest_lr)


val_interval = 1
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()

In [ ]:
for epoch in range(num_epochs):
        
    print("-" * 10)
    print(f"epoch {epoch}/{num_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_data in tqdm(train_loader):
        step += 1
        inputs, labels = batch_data[0].to(device).float(), batch_data[1].to(device).long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_len = len(trainset) // train_loader.batch_size
        if step % 100 == 0:
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}",flush = True)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            metric_sum = 0.0
            metric_count = 0
            val_images = None
            val_labels = None
            val_outputs = None
            val_step = 0
            val_epoch_loss = 0
            dice_vals = list()
            
            for val_data in tqdm(valid_loader):
                val_step += 1
                val_images, val_labels = val_data[0].to(device).float(), val_data[1].to(device).long()
                val_outputs = model(val_images)
                val_labels_list = decollate_batch(val_labels)
                val_labels_convert = [
                post_label(val_label_tensor) for val_label_tensor in val_labels_list
                ]
                val_outputs_list = decollate_batch(val_outputs)
                val_output_convert = [
                    post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list
                ]
                dice_metric(y_pred=val_output_convert, y=val_labels_convert)
                dice = dice_metric.aggregate().item()
                dice_vals.append(dice)
#                 val_loss = loss_function(val_outputs, val_labels)
#                 val_epoch_loss += loss.item()
#                 val_epoch_len = len(validset) // valid_loader.batch_size
            
            metric = np.mean(dice_vals)
#             val_epoch_loss /= val_step
#             metric = val_epoch_loss
#             metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                ppath = Path(os.path.join(pth_dir, f"{submission_name.split('.')[0]}/pre-best_model.pth"))
                ppath.parent.mkdir(parents = True, exist_ok = True)
                torch.save(model.state_dict(), str(ppath))
                print("saved new best metric model")
            print(
                "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                    epoch + 1, metric, best_metric, best_metric_epoch
                )
            )
    if epoch > 7:
        ppath = Path(os.path.join(pth_dir, f"{submission_name.split('.')[0]}/best_model_e{epoch}.pth"))
        ppath.parent.mkdir(parents = True, exist_ok = True)
        torch.save(model.state_dict(), str(ppath))
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

----------
epoch 0/10


  0%|          | 0/517 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 19%|█▉        | 99/517 [02:21<07:55,  1.14s/it]

100/516, train_loss: 0.1554


 38%|███▊      | 199/517 [04:43<08:52,  1.68s/it]

200/516, train_loss: 0.2191


...

500/516, train_loss: 0.1874


100%|██████████| 517/517 [03:17<00:00,  2.62it/s]


epoch 10 average loss: 0.1592


100%|██████████| 130/130 [00:19<00:00,  6.73it/s]


saved new best metric model
current epoch: 10 current mean dice: 0.5049 best mean dice: 0.5049 at epoch 10
train completed, best_metric: 0.5049 at epoch: 10
